In [32]:
#Importing libraries
import json
from langchain.llms import GooglePalm

#Setting up googlepalm and mysql API
with open("keys.json") as f:
    keys = json.load(f)

api_key = keys["api_key"]
llm = GooglePalm(google_api_key=api_key, temperature=0.2)




In [43]:
advice = llm("whats meaning of life in 1 sentense words")
print(advice)

To live and to love.


In [25]:
from langchain.utilities import SQLDatabase

# Extract the database credentials
db_user = keys['db_user']
db_password = keys['db_password']
db_host = keys['db_host']
db_name = keys['db_name']

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [26]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many white t-shirts do we have left for nike in extra small size and white color?")





> Entering new SQLDatabaseChain chain...
How many white t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(40,)]
Answer:40
> Finished chain.


In [27]:
qns1


{'query': 'How many white t-shirts do we have left for nike in extra small size and white color?',
 'result': '40'}

In [28]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts"
 + "WHERE  size = 'S' ")
print(qns2)



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirtsWHERE  size = 'S' 
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('20972'),)]
Answer:20972
> Finished chain.
20972


In [29]:
qns3 = db_chain.run(
    "SELECT ROUND(SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)), 2) AS total_revenue " +
    "FROM (SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id " +
    "FROM t_shirts " +
    "WHERE brand = 'Levi' " +
    "GROUP BY t_shirt_id) a " +
    "LEFT JOIN discounts " +
    "ON a.t_shirt_id = discounts.t_shirt_id"
)
print(qns3)



> Entering new SQLDatabaseChain chain...
SELECT ROUND(SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)), 2) AS total_revenue FROM (SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLQuery:SELECT ROUND(SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)), 2) AS total_revenue FROM (SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('23643.00'),)]
Answer:23643.00
> Finished chain.
23643.00


In [30]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('23811'),)]
Answer:23811
> Finished chain.


In [31]:
qns5 = db_chain.run("How many white color levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(27,), (67,), (82,), (25,)]
Answer:27
> Finished chain.


### Few Shot Learning

In [74]:
few_shots = [
    {
        'Question': "How many white t-shirts do we have left for Nike in extra small size and white color?",
        'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns1
    },
    {
        'Question': "What is the total price of all small size t-shirts?",
        'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns2
    },
    {
        'Question': "If we have to sell all the Levi's T-shirts with discount applied, how much revenue we would make?",
        'SQLQuery': """SELECT ROUND(SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)), 2) AS total_revenue 
                    FROM (SELECT SUM(price * stock_quantity) AS total_amount t_shirt_id 
                    FROM t_shirts 
                    WHERE brand = 'Levi' 
                    GROUP BY t_shirt_id) a 
                    LEFT JOIN discounts 
                    ON a.t_shirt_id = discounts.t_shirt_id""",
        'SQLResult': "Result of the SQL query",
        'Answer': qns3
    },
    {
        'Question': "If we have to sell all the Levi's T-shirts, how much revenue would we make?",
        'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns4
    },
    {
        'Question': "How many white color Levi's shirt I have?",
        'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
        'SQLResult': "Result of the SQL query",
        'Answer': qns5
    }
]

In [76]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [77]:
[" ".join(example.values()) for example in few_shots]
# to_vectorize = [" ".join(str(value) for value in example.values()) for example in few_shots]


TypeError: sequence item 3: expected str instance, dict found

In [64]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/guides/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/deployment/migration#migration-to-0.4.16---november-7,-2023 
